# Import everything


In [1]:
from typing import Tuple
from os.path import exists
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from settings import cfg
from helper import boilerplate, debug, loader
import export_result


Initialized


# Get the dataset

In [2]:
dataset_image, dataset_label = loader.load_data(cfg.train_paths, mmap_mode='c')
punching = dataset_label.sum()
not_punching = dataset_label.shape[0] - punching
print(f'''Stats:
    | Number of punching: {punching}
    | Number of not-punching: {not_punching}''')


Stats:
    | Number of punching: 6355
    | Number of not-punching: 34006


# Boilerplate Code

In [3]:
Data = boilerplate.TrainingDataset


# Model

In [4]:
from settings.cfg import device, model, transforms, optimizer, batch_size, get_idx_gen

criterion = cfg.get_loss([not_punching, punching])
debug.print_model_size(model)


Param size: 512.227MB
Buffer size: 0.032MB


# Train

In [5]:
from sklearn.metrics import f1_score


def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    model.train()
    train = Data(dataset_image, dataset_label, train_idx)
    train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
    total_loss_train = 0

    prediction_array = []
    label_array = []
    for image, label in tqdm(train_dataloader):
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.uint8)

        image = transforms(image)
        # debug.print_image(image)
        output = model(image)

        batch_loss = criterion(output, label)
        total_loss_train += batch_loss.item()

        prediction = output.argmax(dim=1)

        prediction_array.append(prediction.cpu().numpy())
        label_array.append(label.cpu().numpy())

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        model.zero_grad()

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accumulate_train = (prediction_array == label_array).sum().item()
    f1_score_train = f1_score(label_array, prediction_array, average='macro')

    return total_loss_train, total_accumulate_train, f1_score_train


In [6]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    model.eval()
    judge = Data(dataset_image, dataset_label, judge_idx)
    judge_dataloader = DataLoader(judge, batch_size=batch_size)
    total_loss_judge = 0

    prediction_array = []
    label_array = []
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.uint8)

            output = model(image)
            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss.item()

            prediction = output.argmax(dim=1)
            prediction_array.append(prediction.cpu().numpy())
            label_array.append(label.cpu().numpy())

    prediction_array = np.concatenate(prediction_array)
    label_array = np.concatenate(label_array)

    total_accumulate_judge = (prediction_array == label_array).sum().item()
    f1_score_judge = f1_score(label_array, prediction_array, average='macro')

    return total_loss_judge, total_accumulate_judge, f1_score_judge

In [7]:
min_judge_loss = float('inf')
max_f1 = 0.0
f1_targ = 0.6
phase2 = False
under_min = 0
last_submit = 0

if exists(cfg.model_path):
    print("Loading model")
    model.load_state_dict(torch.load(cfg.model_path))

print("Starting phase 1")

for epoch, (train_idx, judge_idx) in enumerate(get_idx_gen().split(dataset_label)):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train, f1_score_train = train(train_idx)
    total_loss_judge, total_accumulate_judge, f1_score_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train/len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Train F1 Score: {f1_score_train:.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}
        | Val F1 Score: {f1_score_judge:.3f}''')

    if f1_score_judge > f1_targ:
        under_min = 0
        max_f1 = f1_score_judge
        break

    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), cfg.model_path)
        print(
            f"Save model because val loss improve loss {min_judge_loss:.3f}")
        under_min = 0
    else:
        under_min += 1
        if under_min > cfg.early_stop:
            print(
                f"Early stop because val loss not improve {under_min} epochs. Resetting model.")
            model.load_state_dict(torch.load(cfg.model_path))
            under_min = 0

print("Starting phase 2")

for epoch, (train_idx, judge_idx) in enumerate(get_idx_gen().split(dataset_label)):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train, f1_score_train = train(train_idx)
    total_loss_judge, total_accumulate_judge, f1_score_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train/len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Train F1 Score: {f1_score_train:.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}
        | Val F1 Score: {f1_score_judge:.3f}''')

    if max_f1 < f1_score_judge:
        if f1_score_judge - last_submit > 0.05:
            last_submit = f1_score_judge
            export_result.submit()

        max_f1 = f1_score_judge
        torch.save(model.state_dict(), cfg.model_path)
        print(f"Save model because val f1 score improve {max_f1:.3f}")
    else:
        under_min += 1
        if under_min > cfg.early_stop:
            print(
                f"Early stop because val f1 score not improve {under_min} epochs. Resetting model.")
            model.load_state_dict(torch.load(cfg.model_path))
            under_min = 0


Loading model
Starting phase 1
Starting epoch 1
        | Train size: 36324
        | Judge size: 4037


  0%|          | 0/1136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

Epoch: 1 
        | Train Loss: 0.016
        | Train Accuracy: 0.759
        | Train F1 Score: 0.664
        | Val Loss: 0.017
        | Val Accuracy: 0.818
        | Val F1 Score: 0.700
Starting phase 2
Starting epoch 1
        | Train size: 36324
        | Judge size: 4037


  0%|          | 0/1136 [00:00<?, ?it/s]

...

Epoch: 8 
        | Train Loss: 0.012
        | Train Accuracy: 0.826
        | Train F1 Score: 0.744
        | Val Loss: 0.019
        | Val Accuracy: 0.872
        | Val F1 Score: 0.756
Save model because val f1 score improve 0.756
Starting epoch 9
        | Train size: 36325
        | Judge size: 4036


  0%|          | 0/1136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

Epoch: 9 
        | Train Loss: 0.012
        | Train Accuracy: 0.833
        | Train F1 Score: 0.752
        | Val Loss: 0.012
        | Val Accuracy: 0.858
        | Val F1 Score: 0.775
Save model because val f1 score improve 0.775
Starting epoch 10
        | Train size: 36325
        | Judge size: 4036


  0%|          | 0/1136 [00:00<?, ?it/s]